In [37]:
# Importing all the necessary libraries in one go to avoid typing manual importing of each module one at a time

from pyforest import *
lazy_imports()

['import plotly.graph_objs as go',
 'import matplotlib.pyplot as plt',
 'import altair as alt',
 'import xgboost as xgb',
 'from sklearn.preprocessing import StandardScaler',
 'from sklearn.linear_model import LassoCV',
 'from pathlib import Path',
 'import plotly as py',
 'from sklearn.linear_model import ElasticNetCV',
 'from PIL import Image',
 'from sklearn.impute import SimpleImputer',
 'from sklearn.ensemble import GradientBoostingClassifier',
 'from sklearn.linear_model import Lasso',
 'import re',
 'from sklearn.model_selection import StratifiedKFold',
 'from sklearn.preprocessing import MinMaxScaler',
 'from sklearn.preprocessing import OneHotEncoder',
 'from sklearn.feature_extraction.text import CountVectorizer',
 'import seaborn as sns',
 'import fbprophet',
 'import bokeh',
 'import imutils',
 'import statistics',
 'import dash',
 'from scipy import stats',
 'from sklearn.feature_extraction.text import TfidfVectorizer',
 'import sys',
 'import tensorflow as tf',
 'from skl

In [38]:
# Importing warnings library to avoid display of warnings prompted during cell executions

import warnings
warnings.filterwarnings("ignore")

In [39]:
# Loading train and test data in separate Dataframes
# We will use this Train_Data Dataframe to check to perform EDA 

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
train_data['source'] = 'train'
test_data['source'] = 'test'
df = pd.concat([train_data, test_data], ignore_index = True)
df.head()

<IPython.core.display.Javascript object>

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [41]:
# For Item Weight, in EDA we saw that the Distribution is Normal
# Hence, replacing the NaN values with Mean

print(f"Mean is : {df['Item_Weight'].mean()}")
df['Item_Weight'].fillna(df['Item_Weight'].mean(), inplace = True)

Mean is : 12.792854228644991


In [42]:
# For Outlet Size, in Data distribution we saw Medium to be the most occuring element for the column
# To avoid significant data loss for model building, if will not be removing the NULL entries rather we will replace missing values with the mode.

print(f"Mode is : {df['Outlet_Size'].mode()}")
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0], inplace = True)

Mode is : 0    Medium
dtype: object


#### For column, Item Outlet Sales, we will have NaN values in the Dataframe as we have merged Train and Test Data and the missing values in column resembles the Test Dataset

In [43]:
# For column Item Visibility, we will saw in our EDA that is 0, which is not possible
# An item will never have 0 visibility
# Chances are that the item was out of stock because of which it was marked as 0
# Let's see the total amount of entries for Items Visibility = 0

print(f"Total Missing values which are marked as 0 in Item Visibility column : {df[df['Item_Visibility'] == 0]['Item_Visibility'].count()}")

# In our EDA we shaw that the Data distribution for Item Visibility was right skewed denoting outliers in our right tail
# We will use Median for imputing the missing values as Mean will be highly impacted with the presence of outlier

df['Item_Visibility'].fillna(df['Item_Visibility'].median(), inplace = True)

Total Missing values which are marked as 0 in Item Visibility column : 879


In [44]:
# Let's check the year range in the Outlet Establishment

print(df['Outlet_Establishment_Year'].value_counts())

# We can see that the range of the year is from 1985 to 2009.
# Let's create a new feature that will contain the total amount of year the Outlet is established and keep it in Outlet_Year

df['Outlet_Years'] = [2009-x if x != 2009 else 1 for x in df['Outlet_Establishment_Year']]

# Let's see the data description
df['Outlet_Years'].describe()

1985    2439
1987    1553
2004    1550
1997    1550
1999    1550
2002    1548
2009    1546
2007    1543
1998     925
Name: Outlet_Establishment_Year, dtype: int64


count    14204.000000
mean        11.278161
std          8.231051
min          1.000000
25%          5.000000
50%         10.000000
75%         22.000000
max         24.000000
Name: Outlet_Years, dtype: float64

In [45]:
# Checking the Item Type column
print(df['Item_Type'].value_counts()[:5])

# Checking the column Item identifier
print(df['Item_Identifier'].value_counts()[:5])

# In our EDA we saw that column Item Identifier are codes that are given to each product for inventory management
# FD : Food, NC: Non Cosumable, DR: Drinks.
# There are categorical data and to use it in model we will have to Encode it
# If each code is take in indiviual then we will be creating huge features which will lead to dimensionality problems
# To avoid it let's take only 3 groups: Food, NC, Drinks
# Extracting each group from the Item Identifier column
df['New_Item_Type'] = df["Item_Identifier"].apply(lambda x: x[:2])

# Rename the new groups for more description
df['New_Item_Type'] = df['New_Item_Type'].map({'FD' : 'Food', 'NC' : 'Non-Consumable', 'DR' : 'Drinks'})

# Let's check the unique values for our extracted column
df['New_Item_Type'].value_counts()

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Name: Item_Type, dtype: int64
FDY02    10
NCK53    10
FDW39    10
FDP33    10
NCA29    10
Name: Item_Identifier, dtype: int64


Food              10201
Non-Consumable     2686
Drinks             1317
Name: New_Item_Type, dtype: int64

In [46]:
# In EDA we saw that our column Item Fat Content have Low Fat, LF, low fat which resembles same as these are naming discrepencies
# Let's use the same name for Low fat for all entries

df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF' : 'Low Fat', 'reg' : 'Regular', 'low fat': 'Low Fat'})

# For the Non Cosumable products the Fat content is given as Regular which is a data error as Non Consumable products donot deal with Fat in data
# We will map Non-Consumable product's FAT content as Non-Edible

df.loc[df['New_Item_Type'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Non-Edible'

# Let's see the unique Item Fat Contents after changes
df['Item_Fat_Content'].value_counts()

Low Fat       6499
Regular       5019
Non-Edible    2686
Name: Item_Fat_Content, dtype: int64

In [47]:
# For Item Visibility column, in general we are aware of the fact that the items which are displayed in front have higher sales
# Let's consider the Hypothesis that the Item Visibility is propotional to sales
# We will create a Pivot table which will contain the Item visibility details with respect to the Item Identifier average

item_visible_avg = df.pivot_table(values = 'Item_Visibility', index = 'Item_Identifier')
item_visible_avg.head()

,Item_Visibility
Item_Identifier,
DRA12,0.034938
DRA24,0.045646
DRA59,0.133384
DRB01,0.079736
DRB13,0.006799


In [48]:
# Creating a lambda function that will generate a column with details of Item Visibility across stores

item_visible_avg_func = lambda x: x['Item_Visibility']/item_visible_avg['Item_Visibility'][item_visible_avg.index == x['Item_Identifier']][0]


# Calling the function to implement the logic of Item Visibility segmentation based on stores
df['Item_Visible_Avg'] = df.apply(item_visible_avg_func, axis = 1).astype(float)

# Checking the dataframe
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Outlet_Years,New_Item_Type,Item_Visible_Avg
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train,10,Food,0.931078
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train,1,Drinks,0.933420
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train,10,Food,0.960069
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,train,11,Food,0.000000
4,NCD19,8.93,Non-Edible,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train,22,Non-Consumable,0.000000


### Next we will be dealing with our Categorical Columns

In [49]:
# We will use Label Encoder for our conversion of categorical columns to Label values

label = LabelEncoder()

# We will create a new varibale for our Outlet Identifier columns
df['Outlet'] = label.fit_transform(df['Outlet_Identifier'])

# Creating list of categorical columns to be encoded
cols_to_encode = ['Item_Fat_Content', 'Outlet_Location_Type', 'Outlet_Size', 'New_Item_Type', 'Outlet_Type', 'Outlet']

# Using a for loop we will iterate over all the columns one by one and transform the data
for i in cols_to_encode:
    df[i] = label.fit_transform(df[i])
    
# Displaying the first 5 columns
df.head()

<IPython.core.display.Javascript object>

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Outlet_Years,New_Item_Type,Item_Visible_Avg,Outlet
0,FDA15,9.30,0,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.1380,train,10,1,0.931078,9
1,DRC01,5.92,2,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.4228,train,1,0,0.933420,3
2,FDN15,17.50,0,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.2700,train,10,1,0.960069,9
3,FDX07,19.20,2,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,1,2,0,732.3800,train,11,1,0.000000,0
4,NCD19,8.93,1,0.000000,Household,53.8614,OUT013,1987,0,2,1,994.7052,train,22,2,0.000000,1


In [50]:
# We have a dataset with Numerical values now but there is a complication in using this
# The conversion of categorical columns to integer types may lead to information misinterpretation since the columns are not Ordinal types
# We will be using One Hot Encoding to segregate the column values with separate entries as a column
# To avoid Dummy trapping we will be removing 1 columns for each encoding as the information can be retrieved by using (n-1) degress of freedom

df = pd.get_dummies(df, columns = cols_to_encode, drop_first = True)
df.head()

<IPython.core.display.Javascript object>

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Outlet_Years,...,Outlet_Type_3,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,10,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,1,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,10,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,11,...,0,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,994.7052,train,22,...,0,1,0,0,0,0,0,0,0,0


In [53]:
# Using the cleaned and transformed dataset for our model building

df.to_csv("C:\\Users\\91983\\Desktop\\Machine Learning\\Ineuron\\Live Projects\\11.BigMartsales\\Self Practise\\Final_data.csv", index = False)